In [1]:
import numpy as np
import pandas as pd

In [21]:
data=pd.read_csv("/content/IMDB Dataset.csv",engine='python',error_bad_lines=False)

In [22]:
data['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [23]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [24]:
data.shape

(50000, 2)

In [25]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [28]:
from nltk.tokenize.toktok import ToktokTokenizer
#Tokenization of text
tokenizers=ToktokTokenizer()
#Setting English stopwords
stopwords=nltk.corpus.stopwords.words('english')

In [34]:
import re,string,unicodedata
from bs4 import BeautifulSoup

In [35]:

#Removing the noisy text
def noiseremoval_text(text):
  soup = BeautifulSoup(text, "html.parser")
  text = soup.get_text()
  text = re.sub('\[[^]]*\]', '', text)
  return text

In [36]:
#Apply function on review column
data['review']=data['review'].apply(noiseremoval_text)

In [37]:
data.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


APPLYING STEMMING

In [38]:
#Stemming the text
def stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

In [40]:
#Apply function on review column
data['review']=data['review'].apply(stemmer)

In [41]:
data.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,A wonder littl production. the film techniqu i...,positive
2,I thought thi wa a wonder way to spend time on...,positive
3,basic there' a famili where a littl boy (jake)...,negative
4,"petter mattei' ""love in the time of money"" is ...",positive


REMOVING STOPWORDS


In [42]:

from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize

In [43]:

#set stopwords to english

stop_wr=set(stopwords.words('english'))
print(stop_wr)

{'to', 'themselves', 'any', 'with', "you're", 'of', "hasn't", 'further', 'don', 'are', "mightn't", 'they', 'and', 'was', 'weren', 'it', 'had', 'what', 'again', 'only', 'be', 'am', 'being', 'its', "that'll", 'against', "you'd", 'y', 'but', 'a', 'ours', 'me', 'same', 'own', 'were', 'after', 'now', 'should', 'up', 'won', 'yourself', 'not', 'the', 'aren', 'doing', 'd', "she's", 'i', "wasn't", 'yourselves', 'll', 'herself', 'other', 'needn', 'yours', 'from', 'mightn', 'why', 'been', "you'll", 'below', 'himself', "mustn't", 'whom', "didn't", 'for', 'each', 'your', 'does', 'above', 'under', 'more', 'if', 'such', 'itself', "isn't", "couldn't", 'about', 'on', 'all', 'an', 'which', 'you', 'who', 'into', 'have', 've', 'off', 're', 'these', 'until', 'that', "should've", "haven't", 'where', 'between', 'her', 'in', 'she', 'his', "hadn't", 'how', 'mustn', 'is', 'during', 'couldn', 'myself', 'this', 'out', 'isn', 'ain', "you've", 'before', 'once', 'ma', 'has', 'shan', 'did', 'or', 'didn', 'when', 'som

In [44]:
#removing the stopwords
def removing_stopwords(text, is_lower_case=False):
    #Tokenization of text
    tokenizers=ToktokTokenizer()
    #Setting English stopwords
    tokens = tokenizers.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filter_tokens = [token for token in tokens if token not in stop_wr]
    else:
        filter_tokens = [token for token in tokens if token.lower() not in stop_wr]
    filtered_text = ' '.join(filter_tokens)    
    return filtered_text

In [45]:
#Apply function on review column
data['review']=data['review'].apply(removing_stopwords)

In [46]:
data.head()

,review,sentiment
0,one review ha mention watch 1 Oz episod ' hook...,positive
1,wonder littl production. film techniqu veri un...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic ' famili littl boy ( jake ) think ' zomb...,negative
4,"petter mattei ' "" love time money "" visual stu...",positive


train and test split


In [48]:
#split the dataset  
#train dataset
train_reviews_data=data.review[:40000]

In [49]:
#test dataset

test_reviews_data=data.review[40000:]

BOW

In [50]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train=cv.fit_transform(train_reviews_data)
#transformed test reviews
cv_test=cv.transform(test_reviews_data)

print('BOW_cv_train:',cv_train.shape)
print('BOW_cv_test:',cv_test.shape)
#vocab=cv.get_feature_names()-toget feature names

BOW_cv_train: (40000, 6369574)
BOW_cv_test: (10000, 6369574)


TF_IDF


In [51]:
#Tfidf vectorizer
tf=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tf_train=tf.fit_transform(train_reviews_data)
#transformed test reviews
tf_test=tf.transform(test_reviews_data)
print('Tfidf_train:',tf_train.shape)
print('Tfidf_test:',tf_test.shape)

Tfidf_train: (40000, 6369574)
Tfidf_test: (10000, 6369574)


LABEL ENCODING

In [52]:
#labeling the sentient data
label=LabelBinarizer()
#transformed sentiment data
sentiment_data=label.fit_transform(data['sentiment'])
print(sentiment_data.shape)

(50000, 1)


In [53]:
sentiment_data

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [54]:
train_data=data.sentiment[:40000]

In [56]:

test_data=data.sentiment[40000:]

In [58]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
#training the model
logistic=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model for Bag of words
lr_bow=logistic.fit(cv_train,train_data)
print(lr_bow)
#Fitting the model for tfidf features
lr_tfidf=logistic.fit(tf_train,train_data)
print(lr_tfidf)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [59]:
#Predicting the model for bag of words
lr_bow_predict=logistic.predict(cv_test)
print(lr_bow_predict)

['negative' 'negative' 'negative' ... 'negative' 'positive' 'positive']


In [60]:
##Predicting the model for tfidf features
lr_tfidf_predict=logistic.predict(tf_test)
print(lr_tfidf_predict)

['negative' 'negative' 'negative' ... 'negative' 'positive' 'positive']


In [62]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
#Accuracy score for bag of words
lr_bow_score=accuracy_score(test_data,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)

lr_bow_score : 0.7522


In [63]:
#Accuracy score for tfidf features
lr_tfidf_score=accuracy_score(test_data,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)

lr_tfidf_score : 0.752
